In [1]:
## Imports

import numpy as np
from custom_envs.gridworlds import WindyGridworldEnv, WindyGridworldEnvNoEncode

from function_approximators.function_approximators import NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics.pairwise import rbf_kernel

from utils.train_utils import train, solve, train_time
from agents.agents import DQNAgent, LinearAgent, FQIAgent, OnlineGaussianProccessAgent
import operator


In [2]:
## Environment

function_approximators = [NeuralNetwork, LinearModel, DecisionTree, RandomForest, SupportVectorRegressor, KNeighboursRegressor, GaussianProcess, OnlineGaussianProcess]
agents = [DQNAgent, LinearAgent, *[FQIAgent]*5, OnlineGaussianProccessAgent]

RENDER = False
env = WindyGridworldEnvNoEncode()

environment = "windygrid"

In [3]:
## Configuration Files

# DQN Config
# 0
CONFIG_DQN = {
    "episode_length": 1000,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.0007,
    "hidden_size": (64,64,16),   # Added additional layer here to compensate for loss in input dim
    "target_update_freq": 200,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "plot_loss": False,
    "epsilon": 1,
    "max_deduct": 0.97,
    "decay": 0.3,
    "lr_step_size": 1000,
    "lr_gamma": 0.95,
    "max_steps": 500,
    "non_param": False,
}

# Linear Config
# 1
CONFIG_LINEAR = {
    "episode_length": 800,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "learning_rate": 0.02,
    "target_update_freq": 50,
    "batch_size": 32,
    "gamma": 0.99,
    "buffer_capacity": int(1e7),
    "plot_loss": False,
    "epsilon": 1,
    "max_steps": 500,
    "poly_degree": 1,
    "max_deduct": 0.97,
    "decay": 0.5,
    "lr_step_size": 1000,
    "lr_gamma": 0.99,
    "non_param": False,
}

# Decision Tree Config
# 2
CONFIG_DT = {
    "episode_length": 500,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 128,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.9,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"criterion":"mse","max_depth": 100, "min_samples_split": 2, "min_samples_leaf": 1},
    "feature_names": ["Cart Position", "Cart Velocity", "Pole Angle", "Pole Angular Velocity", "Action: Push Left", "Action: Push Right"],
    "plot_name": "dt_depth=8",
}

# Random Forest Config
# 3
CONFIG_RF = {
    "episode_length": 500,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 5000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 128,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.9,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"n_estimators": 5,"max_depth": 100, "min_samples_split": 2, "min_samples_leaf": 1},
}

# Support Vector Regressor Config
# 4
CONFIG_SVR = {
    "episode_length": 800, #500
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 64,
    "gamma": 0.99,
    "buffer_capacity": 512,
    "epsilon": 1,
    "max_deduct": 0.95,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"kernel":"rbf", "degree": 3, "C": 2, "gamma" : 1},
}


# K-Neighbors Regressor Config
# 5
CONFIG_KNR = {
    "episode_length": 500,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 1,
    "batch_size": 64,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.9,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"n_neighbors":5, "weights": "distance", "algorithm": "auto", "leaf_size": 30},
}

# Gaussian Process Config
# 6
CONFIG_GP = {
    "episode_length": 500,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "model_save_freq": 1000,
    "model_save_capacity": 20,
    "update_freq": 10,
    "batch_size": 64,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "epsilon": 1,
    "max_deduct": 0.9,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"alpha": 1e-10, "normalize_y": False, "kernel":  RBF(length_scale=0.5, length_scale_bounds="fixed")},
}

# Online Gaussian Process Config
# 7
CONFIG_GP_Online = {
    "episode_length": 500,
    "max_timesteps": 20000,
    "max_time": 30 * 60,
    "eval_freq": 1000, 
    "eval_episodes": 10,
    "gamma": 0.99,
    "buffer_capacity": int(1e6),
    "batch_size": 32,
    "epsilon": 1,
    "max_deduct": 0.9,
    "decay": 0.4,
    "max_steps": 500,
    "non_param": True,
    "model_params": {"sigma_0": 0.5, "init":-200, "kernel":  rbf_kernel, "epsilon_tol": 0.045, "basis_limit": 1000},
}

CONFIGS = [CONFIG_DQN, CONFIG_LINEAR, CONFIG_DT, CONFIG_RF, CONFIG_SVR, CONFIG_KNR, CONFIG_GP, CONFIG_GP_Online]
onlines = [False, False, False, False, False, False, False, True]
models = ["Neural Network", "Linear Model", "Decision Tree", "Random Forest", "Support Vectors", "K-Neighbours", "Gaussian Process", "Gaussian Process Online"]

In [4]:
## Performance Evaluation
env = WindyGridworldEnvNoEncode()
#env = WindyGridworldEnv()



returns = []
train_returns = []
train_times = []
n_seeds=1

j=6
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    r, _, t, times = train(env, 
            CONFIGS[j], 
            fa=function_approximators[j], 
            agent = agents[j], 
            render=RENDER,
            online=onlines[j],
            threshold=0)
    env.close()
    returns.append(r)
    train_returns.append(t)
    train_times.append(times)
    



 Run: 1 



  0%|          | 0/20000 [00:00<?, ?it/s]

Evaluation at timestep 1000 returned a mean returns of -500.0
Epsilon = 0.94375
Replay Buffer count: 128
Evaluation at timestep 2000 returned a mean returns of -500.0
Epsilon = 0.83125
Replay Buffer count: 159
Evaluation at timestep 3000 returned a mean returns of -500.0
Epsilon = 0.71875
Replay Buffer count: 165
Evaluation at timestep 4000 returned a mean returns of -500.0
Epsilon = 0.60625
Replay Buffer count: 177
Evaluation at timestep 5000 returned a mean returns of -500.0
Epsilon = 0.49375
Replay Buffer count: 185
Evaluation at timestep 6000 returned a mean returns of -500.0
Epsilon = 0.38125
Replay Buffer count: 191
JUST FOUND THE END!!!!
Episode Timestep =  170
Evaluation at timestep 7171 returned a mean returns of -500.0
Epsilon = 0.24951249999999991
Replay Buffer count: 199
Evaluation at timestep 8171 returned a mean returns of -500.0
Epsilon = 0.13701249999999998
Replay Buffer count: 204
Evaluation at timestep 9171 returned a mean returns of -500.0
Epsilon = 0.099999999999999

KeyboardInterrupt: 

In [ ]:
with open(f'windygrid_eval_{models[j]}.csv', 'ab') as eval:
    for i in range(n_seeds):
        np.savetxt(eval, [returns[i]], delimiter=',')

In [ ]:
with open(f'windygrid_train_{models[j]}.csv', 'ab') as train:
    for i in range(n_seeds):
        np.savetxt(train, [train_returns[i]], delimiter=',')
        np.savetxt(train, [train_times[i]], delimiter=',')

In [ ]:
## Sample Efficiency Evaluation

n_eps = []
n_steps = []
not_solved = []
n_seeds=30

j=3
for i in range(n_seeds):
    print(f"\n Run: {i+1} \n")
    s, e, n = solve(env, 
            CONFIGS[j], 
            fa=function_approximators[j], 
            agent = agents[j],
            target_return=-16,
            op=operator.ge, 
            render=RENDER,
            online=onlines[j],
            threshold=0)
    env.close()
    n_eps.append(e)
    n_steps.append(s)
    not_solved.append(n)


 Run: 1 

Ep. timesteps: 16
Total timesteps: 2135
Total episodes: 6
Evaluation mean return: -15.9999

 Run: 2 

Ep. timesteps: 15
Total timesteps: 4444
Total episodes: 10
Evaluation mean return: -14.999900000000002

 Run: 3 

Ep. timesteps: 16
Total timesteps: 3372
Total episodes: 8
Evaluation mean return: -15.9999

 Run: 4 

Ep. timesteps: 16
Total timesteps: 4397
Total episodes: 10
Evaluation mean return: -15.9999

 Run: 5 


 Run: 6 


 Run: 7 

Ep. timesteps: 16
Total timesteps: 1966
Total episodes: 5
Evaluation mean return: -15.9999

 Run: 8 

Ep. timesteps: 16
Total timesteps: 1745
Total episodes: 4
Evaluation mean return: -15.9999

 Run: 9 

Ep. timesteps: 16
Total timesteps: 2573
Total episodes: 6
Evaluation mean return: -15.9999

 Run: 10 

Ep. timesteps: 16
Total timesteps: 1587
Total episodes: 4
Evaluation mean return: -15.9999

 Run: 11 

Ep. timesteps: 15
Total timesteps: 1629
Total episodes: 4
Evaluation mean return: -14.999900000000002

 Run: 12 

Ep. timesteps: 16
Tota

In [ ]:
with open(f'{environment}_se_{models[j]}.csv', 'ab') as se:
    np.savetxt(se, [n_eps], delimiter=',')
    np.savetxt(se, [n_steps], delimiter=',')
    np.savetxt(se, [not_solved], delimiter=',')

In [ ]:
mean_eps = np.mean(n_eps)
std_eps = np.std(n_eps)
print(f"Average n_eps: {mean_eps}")
print(f"Std n_eps: {std_eps}")
print(f"St.error n_eps: {std_eps/np.sqrt(n_seeds)}")

mean_steps = np.mean(n_steps)
std_steps = np.std(n_steps)
print(f"Average n_steps: {mean_steps}0")
print(f"Std n_steps: {std_steps}")
print(f"St.error n_steps: {std_steps/np.sqrt(n_seeds)}")

print(f"Not solved: {np.sum(not_solved)} runs")

Average n_eps: 21.633333333333333
Std n_eps: 27.45721924902245
St.error n_eps: 5.0129794496848845
Average n_steps: 900.60
Std n_steps: 1383.9435826651315
St.error n_steps: 252.67237284673604
Not solved: 3 runs


In [ ]:
## Training time

times = []
for j in range(2,8):
        time = train_time(env, 
                CONFIGS[j], 
                fa=function_approximators[j], 
                agent = agents[j],
                online=onlines[j],
                threshold=0)
        env.close()
        times.append(time)

print(time)

20466it [00:33, 608.90it/s]
  0%|          | 0/20000 [00:00<?, ?it/s]

-500


  0%|          | 0/20000 [00:00<?, ?it/s]

-500


  0%|          | 0/20000 [00:00<?, ?it/s]

...

-500


20130it [00:25, 785.00it/s]
  0%|          | 0/20000 [00:00<?, ?it/s]

-499.9996000000001


100%|██████████| 20000/20000 [04:19<00:00, 77.06it/s]

-500
259.5355157852173


In [ ]:
with open(f'{environment}_times.csv', 'ab') as t:
    np.savetxt(t, [times], delimiter=',')